In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm


In [4]:
path_test= r'C:\Users\himanshu.verma\vs_code\POC_MGC\Dataset\AprMayJun23.csv'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_test = pd.read_csv(path_test)
df_test.head(3)

,YEAR,QUARTER,QUARTER_ID,CATEGORY,COUNTRY,CLUSTER_NAME,ACTUAL_BUDGET_FLAG,MONTH,MONTH_ID,ENTITY_NAME,SKU_CODE,MANUFACTURING_OR_TRADE,SKU_DESCRIPTION,CLUSTER_CATEGORY,CLUSTER_SUB_CATEGORY,CLUSTER_CHANNEL,CLUSTER_SUB_CHANNEL,CLUSTER_COUNTRY_ID,CLUSTER_COUNTRY,CLUSTER_REGION,CUSTOMER,CURRENCY,PRODUCT,SUB_PRODUCT,BRAND,SUB_BRAND,TEMPERATURE,VOLUME_MT_ACT,VOLUME_CASES_ACT,GROSS_SALES_ACT,NET_REVENUE_ACT,GROSS_PROFIT_ACT,NET_PROFIT_ACT,CUSTOMER_CODE,SKU_CODE_NEW,DATA_SOURCE,INVENTORY_ITEM_ID,SITE_USE_ID
0,2023,Q2,20232.0,Snacks,Egypt,Snacks,Actual,Jun,202306.0,Senyorita,4083018058,Manufacturing,Windows CHEESE 18 pack * 16 gm* 2 LE.,Undefined,Extruded,Wholesale,Wholesale,NaN,Egypt,Undefined,???????? ??? ????----??? ??????,USD,Extruded Puff,Cheese,Windows,Windows,Ambient,0.007200,25.0,23.134149,21.450040,8.040538,3.271320,9168,4083018058,DIM_ITEM_NEW,837264.0,NaN
1,2023,Q2,20232.0,Snacks,Egypt,Snacks,Actual,Jun,202306.0,Senyorita,4082824009,Manufacturing,CHICKA KETCHUP 30 pack * 10 gm * 1 LE.,Undefined,Extruded,Retail,Grocery/Retail,NaN,Egypt,Undefined,????? ???? ????-----??? ??????,USD,Extruded Corn,Ketchup,Chicka,Chicka,Ambient,0.034500,115.0,88.680852,82.225104,14.004365,-1.945457,10206,4082824009,DIM_ITEM_NEW,676333.0,NaN
2,2023,Q2,20232.0,Snacks,Egypt,Snacks,Actual,Jun,202306.0,Senyorita,4082617059,Manufacturing,Break Vegetable 10 pack * 48 gm * 5LE.,Undefined,Extruded,Wholesale,Wholesale,NaN,Egypt,Undefined,????? ?????? ??? ????? ????----??????,USD,Extruded Pellet,Vegetables,Break,Break,Ambient,0.168003,350.0,450.727750,417.915875,102.387225,13.348980,10320,4082617059,DIM_ITEM_NEW,707338.0,NaN


In [5]:
df_test.isnull().sum()

YEAR                           0
QUARTER                        0
QUARTER_ID                     1
CATEGORY                       1
COUNTRY                        1
CLUSTER_NAME                   1
ACTUAL_BUDGET_FLAG             1
MONTH                          1
MONTH_ID                       1
ENTITY_NAME                    1
SKU_CODE                       1
MANUFACTURING_OR_TRADE         1
SKU_DESCRIPTION                1
CLUSTER_CATEGORY               1
CLUSTER_SUB_CATEGORY           1
CLUSTER_CHANNEL                1
CLUSTER_SUB_CHANNEL            1
CLUSTER_COUNTRY_ID        276214
CLUSTER_COUNTRY                1
CLUSTER_REGION                 1
CUSTOMER                       1
CURRENCY                       1
PRODUCT                        1
SUB_PRODUCT                    1
BRAND                          1
SUB_BRAND                      1
TEMPERATURE                   16
VOLUME_MT_ACT                  1
VOLUME_CASES_ACT               1
GROSS_SALES_ACT                1
NET_REVENU

In [6]:
df_nan_test = df_test.copy()

df_nan_test = df_nan_test.dropna(subset= ['MONTH_ID'])
df_nan_test.isnull().sum()

YEAR                           0
QUARTER                        0
QUARTER_ID                     0
CATEGORY                       0
COUNTRY                        0
CLUSTER_NAME                   0
ACTUAL_BUDGET_FLAG             0
MONTH                          0
MONTH_ID                       0
ENTITY_NAME                    0
SKU_CODE                       0
MANUFACTURING_OR_TRADE         0
SKU_DESCRIPTION                0
CLUSTER_CATEGORY               0
CLUSTER_SUB_CATEGORY           0
CLUSTER_CHANNEL                0
CLUSTER_SUB_CHANNEL            0
CLUSTER_COUNTRY_ID        276213
CLUSTER_COUNTRY                0
CLUSTER_REGION                 0
CUSTOMER                       0
CURRENCY                       0
PRODUCT                        0
SUB_PRODUCT                    0
BRAND                          0
SUB_BRAND                      0
TEMPERATURE                   15
VOLUME_MT_ACT                  0
VOLUME_CASES_ACT               0
GROSS_SALES_ACT                0
NET_REVENU

In [7]:
df_nan_test.shape

(276213, 38)

In [8]:
df_prep_test = df_nan_test[['YEAR', 'MONTH','QUARTER', 'MONTH_ID','SKU_CODE_NEW', 'COUNTRY' ,'CATEGORY', 'CLUSTER_NAME','CLUSTER_COUNTRY', 'CLUSTER_CHANNEL', 
                   'CLUSTER_SUB_CHANNEL', 'VOLUME_MT_ACT', 'NET_REVENUE_ACT', 'GROSS_PROFIT_ACT']]
    
# df_year = df[(df['COUNTRY'] == country) & (df['CLUSTER_NAME'] == cluster_name)]
# df_year.groupby(['MONTH'])[['NET_REVENUE_ACT', 'GROSS_PROFIT_ACT']].sum()
# df_year = df_year[['YEAR', 'MONTH','MONTH_ID', 'COUNTRY', 'CLUSTER_NAME', 'NET_REVENUE_ACT', 'GROSS_PROFIT_ACT']]

df_prep_test['month'] = df_prep_test['MONTH_ID'].astype('str').str[-4:6]
df_prep_test['month'] = df_prep_test['month'].astype('int')

df_prep_test['DATE'] = pd.to_datetime(df_prep_test[['YEAR', 'month']].assign(day=1))
df_prep_test['DATE'] = df_prep_test['DATE'] + pd.offsets.MonthEnd()
df_prep_test.set_index(['DATE'], inplace= True)

## Top 80 SKU's

In [9]:
path_top_80_test = r'C:\Users\himanshu.verma\Downloads\Americana_POC\GS_80Per.xlsx'

df_top_80_test = pd.read_excel(path_top_80_test)
df_top_80_test.columns = ['SKU_CODE_NEW', 'CLUSTER_NAME', 'CLUSTER_COUNTRY', 'CLUSTER_SUB_CHANNEL', 'R_CATEGORY']
df_top_80_test.head(3)

,SKU_CODE_NEW,CLUSTER_NAME,CLUSTER_COUNTRY,CLUSTER_SUB_CHANNEL,R_CATEGORY
0,AGE,AGE,Out of Scope,Out of Scope,Out of Scope
1,4094026036,FF,Egypt,AM Restaurants,Frozen Vegetables
2,4094013020,FF,Egypt,Distributor,Frozen Vegetables


In [10]:
top_80_sku_test = list(df_top_80_test['SKU_CODE_NEW'].unique())
len(top_80_sku_test)

2043

In [11]:
# Selecting only top 80 SKU'set

df_top_80_prep_test = df_prep_test[df_prep_test['SKU_CODE_NEW'].isin(top_80_sku_test)]
df_top_80_prep_test.head()

,YEAR,MONTH,QUARTER,MONTH_ID,SKU_CODE_NEW,COUNTRY,CATEGORY,CLUSTER_NAME,CLUSTER_COUNTRY,CLUSTER_CHANNEL,CLUSTER_SUB_CHANNEL,VOLUME_MT_ACT,NET_REVENUE_ACT,GROSS_PROFIT_ACT,month
DATE,,,,,,,,,,,,,,,
2023-06-30,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.007200,21.450040,8.040538,6
2023-06-30,2023,Jun,Q2,202306.0,4082824009,Egypt,Snacks,Snacks,Egypt,Retail,Grocery/Retail,0.034500,82.225104,14.004365,6
2023-06-30,2023,Jun,Q2,202306.0,4082617059,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.168003,417.915875,102.387225,6
2023-06-30,2023,Jun,Q2,202306.0,4082617059,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.033601,83.583055,20.477325,6
2023-06-30,2023,Jun,Q2,202306.0,4083024050,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.018000,42.985665,8.506958,6


In [12]:
# Adding 'R Category' from ITEM MASTER

path_itm_mstr_test = r'C:\Users\himanshu.verma\Downloads\Americana_POC\ItemMaster.xlsx'

df_itm_mstr_test = pd.read_excel(path_itm_mstr_test)
df_itm_mstr_test = df_itm_mstr_test[['ITEM_CODE', 'R_CATEGORY']]
df_itm_mstr_test.columns = ['SKU_CODE_NEW', 'R_CATEGORY']
df_itm_mstr_test.head()

,SKU_CODE_NEW,R_CATEGORY
0,1000111001,Cake
1,1000111005,Unspecified
2,1000111006,Unspecified
3,1000111007,Unspecified
4,1000111010,Unspecified


In [13]:
# Merging the 'top_80_skus' with 'R category'

df_merge_test = df_top_80_prep_test.merge(df_itm_mstr_test, on= 'SKU_CODE_NEW', how= 'inner')
df_merge_test.head()

,YEAR,MONTH,QUARTER,MONTH_ID,SKU_CODE_NEW,COUNTRY,CATEGORY,CLUSTER_NAME,CLUSTER_COUNTRY,CLUSTER_CHANNEL,CLUSTER_SUB_CHANNEL,VOLUME_MT_ACT,NET_REVENUE_ACT,GROSS_PROFIT_ACT,month,R_CATEGORY
0,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.007200,21.450040,8.040538,6,Snacks
1,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.007200,21.450040,8.040538,6,Snacks
2,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.000288,0.857954,0.321573,6,Snacks
3,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.014400,42.899780,16.080776,6,Snacks
4,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.018720,55.769684,20.904978,6,Snacks


In [14]:
df_merge_test['month'] = df_merge_test['MONTH_ID'].astype('str').str[-4:6]
df_merge_test['month'] = df_merge_test['month'].astype('int')

df_merge_test['DATE'] = pd.to_datetime(df_merge_test[['YEAR', 'month']].assign(day=1))
df_merge_test['DATE'] = df_merge_test['DATE'] + pd.offsets.MonthEnd()
df_merge_test.head()

,YEAR,MONTH,QUARTER,MONTH_ID,SKU_CODE_NEW,COUNTRY,CATEGORY,CLUSTER_NAME,CLUSTER_COUNTRY,CLUSTER_CHANNEL,CLUSTER_SUB_CHANNEL,VOLUME_MT_ACT,NET_REVENUE_ACT,GROSS_PROFIT_ACT,month,R_CATEGORY,DATE
0,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.007200,21.450040,8.040538,6,Snacks,2023-06-30
1,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.007200,21.450040,8.040538,6,Snacks,2023-06-30
2,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.000288,0.857954,0.321573,6,Snacks,2023-06-30
3,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.014400,42.899780,16.080776,6,Snacks,2023-06-30
4,2023,Jun,Q2,202306.0,4083018058,Egypt,Snacks,Snacks,Egypt,Wholesale,Wholesale,0.018720,55.769684,20.904978,6,Snacks,2023-06-30


In [15]:
# Taking absolute values of Volume

# df_merge_test['VOLUME_MT_ACT'] = abs(df_merge_test['VOLUME_MT_ACT'])

In [16]:
# Grouping on various levels (4 channels)

df_grouped_test = df_merge_test.groupby(['CLUSTER_NAME', 'CLUSTER_CHANNEL', 'CLUSTER_COUNTRY', 'R_CATEGORY','month', 'YEAR', 'SKU_CODE_NEW'])['VOLUME_MT_ACT'].sum()
df_grouped_test_res = df_grouped_test.reset_index()
df_grouped_test_res.head()

,CLUSTER_NAME,CLUSTER_CHANNEL,CLUSTER_COUNTRY,R_CATEGORY,month,YEAR,SKU_CODE_NEW,VOLUME_MT_ACT
0,FF,3rd party sales,Egypt,Frozen Vegetables,4,2023,4094026612,861.00
1,FF,3rd party sales,Egypt,Frozen Vegetables,5,2023,4094026612,826.50
2,FF,3rd party sales,Egypt,Frozen Vegetables,6,2023,4094026612,720.00
3,FF,3rd party sales,Exports & Private Label,Frozen Vegetables,4,2023,4094026643,413.82
4,FF,3rd party sales,Exports & Private Label,Frozen Vegetables,4,2023,4094026673,84.00


In [17]:
df_grouped_test[:20]

CLUSTER_NAME  CLUSTER_CHANNEL  CLUSTER_COUNTRY          R_CATEGORY         month  YEAR  SKU_CODE_NEW
FF            3rd party sales  Egypt                    Frozen Vegetables  4      2023  4094026612       861.0000
                                                                           5      2023  4094026612       826.5000
                                                                           6      2023  4094026612       720.0000
                               Exports & Private Label  Frozen Vegetables  4      2023  4094026643       413.8200
                                                                                        4094026673        84.0000
                                                                           5      2023  4094026643       522.7200
                                                                           6      2023  4094026643       827.6400
                                                                                        4094026672   

In [18]:
test_cluster_name_list = list(df_grouped_test_res['CLUSTER_NAME'].unique())
test_cluster_country_list = list(df_grouped_test_res['CLUSTER_COUNTRY'].unique())
test_cluster_channel_list = list(df_grouped_test_res['CLUSTER_CHANNEL'].unique())
test_cluster_r_category_list = list(df_grouped_test_res['R_CATEGORY'].unique())
test_cluster_sku_list = list(df_grouped_test_res['SKU_CODE_NEW'].unique())

# Unique values in all lists

print(test_cluster_name_list, len(test_cluster_name_list))
print(test_cluster_country_list, len(test_cluster_country_list))
print(test_cluster_channel_list, len(test_cluster_channel_list))
print(test_cluster_r_category_list, len(test_cluster_r_category_list))
print(test_cluster_sku_list, len(test_cluster_sku_list))

['FF', 'GFI', 'ICAPP', 'KSA Cake', 'Kwt Agencies', 'Kwt Bakeries', 'Meat', 'Snacks'] 8
['Egypt', 'Exports & Private Label', 'Lower Gulf', 'KSA', 'Kuwait', 'Supply Chain Led'] 6
['3rd party sales', 'Distributor', 'E-Commerce', 'Food Services', 'ICP AM Foods', 'Intl sales', 'Retail', 'Others', 'Industrial Sales', 'Wholesale'] 10
['Frozen Vegetables', 'C&W', 'Cake', 'Kwt Agencies', 'Bakeries', 'Protein', 'Frozen Veg-Meat', 'Snacks'] 8
['4094026612', '4094026643', '4094026673', '4094026672', '4094000099', '4094013020', '4094015078', '4094015520', '4094016015', '4094023020', '4094025520', '4094026015', '4094026033', '4094033020', '4094036015', '4094135015', '4094630530', '4094026034', '4094026036', '4094026039', '4094676016', '4094026037', '4094026040', '4094676040', '4094026027', '4094010401', '4094014604', '4094014606', '4094014607', '4094014623', '4094030401', '4094034603', '4094034604', '4094034605', '4094090401', '4094015025', '4094015049', '4094010912', '4094030912', '4094060812', '40

## Filtering by 
1. Cluster_name = 'Meat'
2. R-category = 'Protein'

In [19]:
df_grouped_res_meat_test = df_grouped_test_res[(df_grouped_test_res['CLUSTER_NAME'] == 'Meat') & (df_grouped_test_res['R_CATEGORY'] == 'Protein')]
df_grouped_res_meat_test.head()

,CLUSTER_NAME,CLUSTER_CHANNEL,CLUSTER_COUNTRY,R_CATEGORY,month,YEAR,SKU_CODE_NEW,VOLUME_MT_ACT
4616,Meat,3rd party sales,Exports & Private Label,Protein,6,2023,4001310178,5.7480
4617,Meat,3rd party sales,Exports & Private Label,Protein,6,2023,4001310181,2.7800
4618,Meat,3rd party sales,Exports & Private Label,Protein,6,2023,4001370195,1.8000
4619,Meat,Distributor,Exports & Private Label,Protein,4,2023,4001370244,5.7600
4620,Meat,Distributor,Exports & Private Label,Protein,4,2023,4001971946,9.9225


In [20]:
cluster_name_list_meat_test = list(df_grouped_res_meat_test['CLUSTER_NAME'].unique())
cluster_country_list_meat_test = list(df_grouped_res_meat_test['CLUSTER_COUNTRY'].unique())
cluster_channel_list_meat_test = list(df_grouped_res_meat_test['CLUSTER_CHANNEL'].unique())
cluster_r_category_list_meat_test = list(df_grouped_res_meat_test['R_CATEGORY'].unique())
cluster_sku_list_meat_test = list(df_grouped_res_meat_test['SKU_CODE_NEW'].unique())

# Unique values in all lists

print(cluster_name_list_meat_test, len(cluster_name_list_meat_test))
print(cluster_country_list_meat_test, len(cluster_country_list_meat_test))
print(cluster_channel_list_meat_test, len(cluster_channel_list_meat_test))
print(cluster_r_category_list_meat_test, len(cluster_r_category_list_meat_test))
print(cluster_sku_list_meat_test, len(cluster_sku_list_meat_test))

['Meat'] 1
['Exports & Private Label', 'Lower Gulf', 'KSA', 'Kuwait'] 4
['3rd party sales', 'Distributor', 'E-Commerce', 'Food Services', 'Intl sales', 'Retail', 'Wholesale'] 7
['Protein'] 1
['4001310178', '4001310181', '4001370195', '4001370244', '4001971946', '4001971947', '4001310103', '4001310134', '4001310140', '4001310141', '4001310149', '4001310174', '4001310182', '4001310184', '4001310185', '4001310187', '4001310402', '4001310409', '4001310505', '4001310601', '4001310701', '4001320101', '4001320114', '4001320201', '4001320210', '4001320217', '4001330103', '4001330203', '4001340501', '4001350114', '4001350115', '4001350121', '4001350125', '4001350128', '4001350207', '4001351001', '4001351006', '4001351016', '4001351022', '4001351112', '4001360101', '4001360106', '4001360117', '4001360132', '4001360307', '4001360308', '4001361602', '4001362302', '4001370106', '4001370119', '4001370124', '4001370125', '4001370134', '4001370135', '4001370140', '4001370144', '4001370145', '400137015

In [21]:
result_df_test= pd.DataFrame()
result_df_test['Date'] = pd.date_range(start='2023-04-01', end='2023-07-01', freq='M')
result_df_test

,Date
0,2023-04-30
1,2023-05-31
2,2023-06-30


In [22]:
counter = 0
total = 0
for i in cluster_name_list_meat_test:
    for j in cluster_country_list_meat_test:
        for k in cluster_channel_list_meat_test:
            for l in cluster_r_category_list_meat_test:
                for m in cluster_sku_list_meat_test:
                    
                    df_single_test = df_grouped_test_res[(df_grouped_test_res['SKU_CODE_NEW'] == m) & (df_grouped_test_res['CLUSTER_NAME'] == i) & (df_grouped_test_res['CLUSTER_CHANNEL'] == k) & (df_grouped_test_res['CLUSTER_COUNTRY'] == j) & (df_grouped_test_res['R_CATEGORY'] == l)]
                    col_name = i + '_' + j + '_' + k + '_' + l + '_' + m
                    df_single_test[col_name] = df_single_test['VOLUME_MT_ACT']
                    df_single_test['Date'] = pd.to_datetime(df_single_test[['YEAR', 'month']].assign(day=1))
                    df_single_test['Date'] = df_single_test['Date'] + pd.offsets.MonthEnd()
                    
                    df_single_test = df_single_test.loc[:, ['Date', col_name]]
                    # print(df_single)
                    total += 1
                    if len(df_single_test)>0:
                        result_df_test = result_df_test.merge(df_single_test, on= 'Date', how= 'left')
                        counter += 1
                    
                    print(counter, total, end='\r')
                
                    
                    

In [23]:
result_df_test.shape

(3, 1151)

In [24]:
pd.set_option('display.max_rows', None)
df_nan_test = pd.DataFrame(result_df_test.isnull().sum() / len(result_df_test) * 100)


# sum(df_nan[0] == 0)
columns_full_test = [col for col in result_df_test.columns if ((result_df_test[col]).isnull().sum() / len(df_nan_test) == 0)]
columns_full_test

['Date',
 'Meat_Exports & Private Label_Distributor_Protein_4001370244',
 'Meat_Exports & Private Label_Distributor_Protein_4001971946',
 'Meat_Exports & Private Label_Distributor_Protein_4001971947',
 'Meat_Exports & Private Label_Food Services_Protein_4001410210',
 'Meat_Exports & Private Label_Food Services_Protein_4001410212',
 'Meat_Exports & Private Label_Food Services_Protein_4001430105',
 'Meat_Exports & Private Label_Food Services_Protein_4001470203',
 'Meat_Exports & Private Label_Food Services_Protein_4001470304',
 'Meat_Exports & Private Label_Food Services_Protein_4001486101',
 'Meat_Exports & Private Label_Intl sales_Protein_4001310601',
 'Meat_Exports & Private Label_Intl sales_Protein_4001350114',
 'Meat_Exports & Private Label_Intl sales_Protein_4001350207',
 'Meat_Exports & Private Label_Intl sales_Protein_4001360101',
 'Meat_Exports & Private Label_Intl sales_Protein_4001370119',
 'Meat_Exports & Private Label_Intl sales_Protein_4001370150',
 'Meat_Exports & Private 

In [25]:
len(columns_full_test)

840

In [26]:
result_df_test.head()

,Date,Meat_Exports & Private Label_3rd party sales_Protein_4001310178,Meat_Exports & Private Label_3rd party sales_Protein_4001310181,Meat_Exports & Private Label_3rd party sales_Protein_4001370195,Meat_Exports & Private Label_Distributor_Protein_4001370244,Meat_Exports & Private Label_Distributor_Protein_4001971946,Meat_Exports & Private Label_Distributor_Protein_4001971947,Meat_Exports & Private Label_Food Services_Protein_4001410210,Meat_Exports & Private Label_Food Services_Protein_4001410212,Meat_Exports & Private Label_Food Services_Protein_4001430105,Meat_Exports & Private Label_Food Services_Protein_4001470103,Meat_Exports & Private Label_Food Services_Protein_4001470128,Meat_Exports & Private Label_Food Services_Protein_4001470203,Meat_Exports & Private Label_Food Services_Protein_4001470304,Meat_Exports & Private Label_Food Services_Protein_4001486101,Meat_Exports & Private Label_Food Services_Protein_4001471502,Meat_Exports & Private Label_Intl sales_Protein_4001310178,Meat_Exports & Private Label_Intl sales_Protein_4001310103,Meat_Exports & Private Label_Intl sales_Protein_4001310134,Meat_Exports & Private Label_Intl sales_Protein_4001310140,Meat_Exports & Private Label_Intl sales_Protein_4001310141,Meat_Exports & Private Label_Intl sales_Protein_4001310184,Meat_Exports & Private Label_Intl sales_Protein_4001310601,Meat_Exports & Private Label_Intl sales_Protein_4001320101,Meat_Exports & Private Label_Intl sales_Protein_4001320201,Meat_Exports & Private Label_Intl sales_Protein_4001330103,Meat_Exports & Private Label_Intl sales_Protein_4001330203,Meat_Exports & Private Label_Intl sales_Protein_4001350114,Meat_Exports & Private Label_Intl sales_Protein_4001350207,Meat_Exports & Private Label_Intl sales_Protein_4001351001,Meat_Exports & Private Label_Intl sales_Protein_4001351006,Meat_Exports & Private Label_Intl sales_Protein_4001351016,Meat_Exports & Private Label_Intl sales_Protein_4001351022,Meat_Exports & Private Label_Intl sales_Protein_4001351112,Meat_Exports & Private Label_Intl sales_Protein_4001360101,Meat_Exports & Private Label_Intl sales_Protein_4001360106,Meat_Exports & Private Label_Intl sales_Protein_4001360117,Meat_Exports & Private Label_Intl sales_Protein_4001370106,Meat_Exports & Private Label_Intl sales_Protein_4001370119,Meat_Exports & Private Label_Intl sales_Protein_4001370124,Meat_Exports & Private Label_Intl sales_Protein_4001370144,Meat_Exports & Private Label_Intl sales_Protein_4001370145,Meat_Exports & Private Label_Intl sales_Protein_4001370150,Meat_Exports & Private Label_Intl sales_Protein_4001370226,Meat_Exports & Private Label_Intl sales_Protein_4001370230,Meat_Exports & Private Label_Intl sales_Protein_4001370231,Meat_Exports & Private Label_Intl sales_Protein_4001370235,Meat_Exports & Private Label_Intl sales_Protein_4001370255,Meat_Exports & Private Label_Intl sales_Protein_4001370316,Meat_Exports & Private Label_Intl sales_Protein_4001370707,Meat_Exports & Private Label_Intl sales_Protein_4001370806,Meat_Exports & Private Label_Intl sales_Protein_4001370834,Meat_Exports & Private Label_Intl sales_Protein_4001370846,Meat_Exports & Private Label_Intl sales_Protein_4001370852,Meat_Exports & Private Label_Intl sales_Protein_4001370904,Meat_Exports & Private Label_Intl sales_Protein_4001370905,Meat_Exports & Private Label_Intl sales_Protein_4001371201,Meat_Exports & Private Label_Intl sales_Protein_4001371504,Meat_Exports & Private Label_Intl sales_Protein_4001371505,Meat_Exports & Private Label_Intl sales_Protein_4001371511,Meat_Exports & Private Label_Intl sales_Protein_4001371512,Meat_Exports & Private Label_Intl sales_Protein_4001371514,Meat_Exports & Private Label_Intl sales_Protein_4001386131,Meat_Exports & Private Label_Intl sales_Protein_4001370860,Meat_Exports & Private Label_Intl sales_Protein_4001371109,Meat_Exports & Private Label_Intl sales_Protein_4001370123,Meat_Exports & Private Label_Intl sales_Protein_4001360102,Meat_Exports & Private Label_Intl sales_Protein_4001370

In [27]:
result_df_test.shape

(3, 1151)

In [28]:
# Selecting 100 % non-null series

df_no_null_test = result_df_test.loc[:, columns_full_test]
df_no_null_test

,Date,Meat_Exports & Private Label_Distributor_Protein_4001370244,Meat_Exports & Private Label_Distributor_Protein_4001971946,Meat_Exports & Private Label_Distributor_Protein_4001971947,Meat_Exports & Private Label_Food Services_Protein_4001410210,Meat_Exports & Private Label_Food Services_Protein_4001410212,Meat_Exports & Private Label_Food Services_Protein_4001430105,Meat_Exports & Private Label_Food Services_Protein_4001470203,Meat_Exports & Private Label_Food Services_Protein_4001470304,Meat_Exports & Private Label_Food Services_Protein_4001486101,Meat_Exports & Private Label_Intl sales_Protein_4001310601,Meat_Exports & Private Label_Intl sales_Protein_4001350114,Meat_Exports & Private Label_Intl sales_Protein_4001350207,Meat_Exports & Private Label_Intl sales_Protein_4001360101,Meat_Exports & Private Label_Intl sales_Protein_4001370119,Meat_Exports & Private Label_Intl sales_Protein_4001370150,Meat_Exports & Private Label_Intl sales_Protein_4001370231,Meat_Exports & Private Label_Intl sales_Protein_4001370707,Meat_Exports & Private Label_Intl sales_Protein_4001370806,Meat_Exports & Private Label_Intl sales_Protein_4001370834,Meat_Exports & Private Label_Intl sales_Protein_4001370852,Meat_Exports & Private Label_Intl sales_Protein_4001370904,Meat_Exports & Private Label_Intl sales_Protein_4001371504,Meat_Exports & Private Label_Intl sales_Protein_4001371511,Meat_Exports & Private Label_Intl sales_Protein_4001371512,Meat_Exports & Private Label_Intl sales_Protein_4001371514,Meat_Exports & Private Label_Intl sales_Protein_4001370860,Meat_Exports & Private Label_Intl sales_Protein_4001360102,Meat_Exports & Private Label_Intl sales_Protein_4001310610,Meat_Exports & Private Label_Intl sales_Protein_4001310160,Meat_Exports & Private Label_Intl sales_Protein_4001350122,Meat_Exports & Private Label_Intl sales_Protein_4001351007,Meat_Exports & Private Label_Intl sales_Protein_4001370265,Meat_Lower Gulf_Distributor_Protein_4001310103,Meat_Lower Gulf_Distributor_Protein_4001310134,Meat_Lower Gulf_Distributor_Protein_4001310140,Meat_Lower Gulf_Distributor_Protein_4001310141,Meat_Lower Gulf_Distributor_Protein_4001310174,Meat_Lower Gulf_Distributor_Protein_4001310184,Meat_Lower Gulf_Distributor_Protein_4001310185,Meat_Lower Gulf_Distributor_Protein_4001310409,Meat_Lower Gulf_Distributor_Protein_4001310601,Meat_Lower Gulf_Distributor_Protein_4001310701,Meat_Lower Gulf_Distributor_Protein_4001320101,Meat_Lower Gulf_Distributor_Protein_4001330103,Meat_Lower Gulf_Distributor_Protein_4001330203,Meat_Lower Gulf_Distributor_Protein_4001340501,Meat_Lower Gulf_Distributor_Protein_4001350125,Meat_Lower Gulf_Distributor_Protein_4001351001,Meat_Lower Gulf_Distributor_Protein_4001351022,Meat_Lower Gulf_Distributor_Protein_4001351112,Meat_Lower Gulf_Distributor_Protein_4001360101,Meat_Lower Gulf_Distributor_Protein_4001360106,Meat_Lower Gulf_Distributor_Protein_4001360117,Meat_Lower Gulf_Distributor_Protein_4001360307,Meat_Lower Gulf_Distributor_Protein_4001360308,Meat_Lower Gulf_Distributor_Protein_4001361602,Meat_Lower Gulf_Distributor_Protein_4001362302,Meat_Lower Gulf_Distributor_Protein_4001370106,Meat_Lower Gulf_Distributor_Protein_4001370119,Meat_Lower Gulf_Distributor_Protein_4001370124,Meat_Lower Gulf_Distributor_Protein_4001370125,Meat_Lower Gulf_Distributor_Protein_4001370144,Meat_Lower Gulf_Distributor_Protein_4001370145,Meat_Lower Gulf_Distributor_Protein_4001370150,Meat_Lower Gulf_Distributor_Protein_4001370172,Meat_Lower Gulf_Distributor_Protein_4001370181,Meat_Lower Gulf_Distributor_Protein_4001370200,Meat_Lower Gulf_Distributor_Protein_4001370211,Meat_Lower Gulf_Distributor_Protein_4001370226,Meat_Lower Gulf_Distributor_Protein_4001370230,Meat_Lower Gulf_Distributor_Protein_4001370231,Meat_Lower Gulf_Distributor_Protein_4001370235,Meat_Lower Gulf_Distributor_Protein_4001370255,Meat_Lower Gulf_Distributor_Protein_4001370274,Meat_Lower Gulf_Distributor_Protein_4001370275,Meat_Lower Gulf_Distributor_Protein_4001370283,Meat_Lower Gulf_D

In [29]:
df_no_null_test.shape

(3, 840)

##### --------------------------------------------------------------------
### Working with train data
##### --------------------------------------------------------------------

In [30]:
train_path = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\Meat_protein_no_nan.csv'

df_train = pd.read_csv(train_path)
df_train.set_index(['Date'], inplace= True)

In [31]:
df_train = df_train.drop(columns= ['Unnamed: 0'])
df_train.head()

,Meat_KSA_Food Services_Protein_4001320101,Meat_KSA_Food Services_Protein_4001330103,Meat_KSA_Food Services_Protein_4001350207,Meat_KSA_Food Services_Protein_4001370144,Meat_KSA_Food Services_Protein_4001370145,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Food Services_Protein_4001371514,Meat_KSA_Food Services_Protein_4001370150,Meat_KSA_Food Services_Protein_4001362302,Meat_KSA_Food Services_Protein_4001371108,Meat_KSA_Food Services_Protein_5011840107,Meat_KSA_Food Services_Protein_4001371508,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Food Services_Protein_4001371509,Meat_KSA_Food Services_Protein_4001370310,Meat_KSA_Food Services_Protein_4001370807,Meat_KSA_Food Services_Protein_4001370140,Meat_KSA_Food Services_Protein_4001350111,Meat_KSA_Food Services_Protein_4001310114,Meat_KSA_Food Services_Protein_4001340101,Meat_KSA_Food Services_Protein_4001340301,Meat_KSA_Food Services_Protein_4001350602,Meat_KSA_Food Services_Protein_4001351012,Meat_KSA_Food Services_Protein_4001370147,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Food Services_Protein_4001370816,Meat_KSA_Food Services_Protein_4001370889,Meat_KSA_Food Services_Protein_4003350305,Meat_KSA_Food Services_Protein_4003350401,Meat_KSA_Retail_Protein_4001310103,Meat_KSA_Retail_Protein_4001310140,Meat_KSA_Retail_Protein_4001310141,Meat_KSA_Retail_Protein_4001320101,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Retail_Protein_4001330103,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001350207,Meat_KSA_Retail_Protein_4001360102,Meat_KSA_Retail_Protein_4001360106,Meat_KSA_Retail_Protein_4001360308,Meat_KSA_Retail_Protein_4001370123,Meat_KSA_Retail_Protein_4001370125,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001370226,Meat_KSA_Retail_Protein_4001370231,Meat_KSA_Retail_Protein_4001370235,Meat_KSA_Retail_Protein_4001370255,Meat_KSA_Retail_Protein_4001370303,Meat_KSA_Retail_Protein_4001370707,Meat_KSA_Retail_Protein_4001370806,Meat_KSA_Retail_Protein_4001370846,Meat_KSA_Retail_Protein_4001370852,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Retail_Protein_4001370905,Meat_KSA_Retail_Protein_4001371109,Meat_KSA_Retail_Protein_4001371201,Meat_KSA_Retail_Protein_4001371514,Meat_KSA_Retail_Protein_4001370150,Meat_KSA_Retail_Protein_4001370306,Meat_KSA_Retail_Protein_4001370316,Meat_KSA_Retail_Protein_4001370904,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Retail_Protein_4001386130,Meat_KSA_Retail_Protein_4001310174,Meat_KSA_Retail_Protein_4001310402,Meat_KSA_Retail_Protein_4001310505,Meat_KSA_Retail_Protein_4001310701,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001360307,Meat_KSA_Retail_Protein_4001361602,Meat_KSA_Retail_Protein_4001362302,Meat_KSA_Retail_Protein_4001370181,Meat_KSA_Retail_Protein_4001370230,Meat_KSA_Retail_Protein_4001370283,Meat_KSA_Retail_Protein_4001370325,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001371108,Meat_KSA_Retail_Protein_4001371537,Meat_KSA_Retail_Protein_4003350501,Meat_KSA_Retail_Protein_5011810203,Meat_KSA_Retail_Protein_5011840107,Meat_KSA_Retail_Protein_4001310149,Meat_KSA_Retail_Protein_4001371508,Meat_KSA_Retail_Protein_4001350115,Meat_KSA_Retail_Protein_4001370845,Meat_KSA_Retail_Protein_4001371538,Meat_KSA_Retail_Protein_4001370310,Meat_KSA_Retail_Protein_4001370807,Meat_KSA_Retail_Protein_4001310123,Meat_KSA_Retail_Protein_4001370284,Meat_KSA_Retail_Protein_4001370285,Meat_KSA_Retail_Protein_4001370140,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Retail_Protein_4001310506,Meat_KSA_Retail_Protein_4001310114,Meat_KSA_Retail_Protein_4001340101,Meat_KSA_Retail_Protein_4001340301,Meat_KSA_Retail_Protein_4001350602,Meat_KSA_Retail_Protein_4001351201,Meat_KSA_Retail_Protein_4001370147,Meat_KSA_Retail_Protein_4001370272,Meat_KSA_Retail_Protein_4001370273,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Retail_Protein_4003350305,Meat_KSA_Retail_Protein_4003350308,Meat_KSA_Retail_Protein_4003350401,Meat_KSA_

In [32]:
## Removing the 2 ID's not present in train data 
# (['Meat_KSA_Retail_Protein_5011680102', 'Meat_KSA_Retail_Protein_5011680103'])

df_train = df_train.drop(columns= ['Meat_KSA_Retail_Protein_5011680102', 'Meat_KSA_Retail_Protein_5011680103'])
df_train.head(2)

,Meat_KSA_Food Services_Protein_4001320101,Meat_KSA_Food Services_Protein_4001330103,Meat_KSA_Food Services_Protein_4001350207,Meat_KSA_Food Services_Protein_4001370144,Meat_KSA_Food Services_Protein_4001370145,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Food Services_Protein_4001371514,Meat_KSA_Food Services_Protein_4001370150,Meat_KSA_Food Services_Protein_4001362302,Meat_KSA_Food Services_Protein_4001371108,Meat_KSA_Food Services_Protein_5011840107,Meat_KSA_Food Services_Protein_4001371508,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Food Services_Protein_4001371509,Meat_KSA_Food Services_Protein_4001370310,Meat_KSA_Food Services_Protein_4001370807,Meat_KSA_Food Services_Protein_4001370140,Meat_KSA_Food Services_Protein_4001350111,Meat_KSA_Food Services_Protein_4001310114,Meat_KSA_Food Services_Protein_4001340101,Meat_KSA_Food Services_Protein_4001340301,Meat_KSA_Food Services_Protein_4001350602,Meat_KSA_Food Services_Protein_4001351012,Meat_KSA_Food Services_Protein_4001370147,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Food Services_Protein_4001370816,Meat_KSA_Food Services_Protein_4001370889,Meat_KSA_Food Services_Protein_4003350305,Meat_KSA_Food Services_Protein_4003350401,Meat_KSA_Retail_Protein_4001310103,Meat_KSA_Retail_Protein_4001310140,Meat_KSA_Retail_Protein_4001310141,Meat_KSA_Retail_Protein_4001320101,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Retail_Protein_4001330103,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001350207,Meat_KSA_Retail_Protein_4001360102,Meat_KSA_Retail_Protein_4001360106,Meat_KSA_Retail_Protein_4001360308,Meat_KSA_Retail_Protein_4001370123,Meat_KSA_Retail_Protein_4001370125,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001370226,Meat_KSA_Retail_Protein_4001370231,Meat_KSA_Retail_Protein_4001370235,Meat_KSA_Retail_Protein_4001370255,Meat_KSA_Retail_Protein_4001370303,Meat_KSA_Retail_Protein_4001370707,Meat_KSA_Retail_Protein_4001370806,Meat_KSA_Retail_Protein_4001370846,Meat_KSA_Retail_Protein_4001370852,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Retail_Protein_4001370905,Meat_KSA_Retail_Protein_4001371109,Meat_KSA_Retail_Protein_4001371201,Meat_KSA_Retail_Protein_4001371514,Meat_KSA_Retail_Protein_4001370150,Meat_KSA_Retail_Protein_4001370306,Meat_KSA_Retail_Protein_4001370316,Meat_KSA_Retail_Protein_4001370904,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Retail_Protein_4001386130,Meat_KSA_Retail_Protein_4001310174,Meat_KSA_Retail_Protein_4001310402,Meat_KSA_Retail_Protein_4001310505,Meat_KSA_Retail_Protein_4001310701,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001360307,Meat_KSA_Retail_Protein_4001361602,Meat_KSA_Retail_Protein_4001362302,Meat_KSA_Retail_Protein_4001370181,Meat_KSA_Retail_Protein_4001370230,Meat_KSA_Retail_Protein_4001370283,Meat_KSA_Retail_Protein_4001370325,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001371108,Meat_KSA_Retail_Protein_4001371537,Meat_KSA_Retail_Protein_4003350501,Meat_KSA_Retail_Protein_5011810203,Meat_KSA_Retail_Protein_5011840107,Meat_KSA_Retail_Protein_4001310149,Meat_KSA_Retail_Protein_4001371508,Meat_KSA_Retail_Protein_4001350115,Meat_KSA_Retail_Protein_4001370845,Meat_KSA_Retail_Protein_4001371538,Meat_KSA_Retail_Protein_4001370310,Meat_KSA_Retail_Protein_4001370807,Meat_KSA_Retail_Protein_4001310123,Meat_KSA_Retail_Protein_4001370284,Meat_KSA_Retail_Protein_4001370285,Meat_KSA_Retail_Protein_4001370140,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Retail_Protein_4001310506,Meat_KSA_Retail_Protein_4001310114,Meat_KSA_Retail_Protein_4001340101,Meat_KSA_Retail_Protein_4001340301,Meat_KSA_Retail_Protein_4001350602,Meat_KSA_Retail_Protein_4001351201,Meat_KSA_Retail_Protein_4001370147,Meat_KSA_Retail_Protein_4001370272,Meat_KSA_Retail_Protein_4001370273,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Retail_Protein_4003350305,Meat_KSA_Retail_Protein_4003350308,Meat_KSA_Retail_Protein_4003350401,Meat_KSA_

In [33]:
# Selecting all ID as train data

# df_train.to_csv('train_all.csv')

### Selecting all ID's of train (200)

In [34]:
df_no_null_ids_list_all = list(df_train.columns)
df_no_null_ids_list_all = df_no_null_ids_list_all
df_no_null_ids_list_all, len(df_no_null_ids_list_all)

(['Meat_KSA_Food Services_Protein_4001320101',
  'Meat_KSA_Food Services_Protein_4001330103',
  'Meat_KSA_Food Services_Protein_4001350207',
  'Meat_KSA_Food Services_Protein_4001370144',
  'Meat_KSA_Food Services_Protein_4001370145',
  'Meat_KSA_Food Services_Protein_4001370861',
  'Meat_KSA_Food Services_Protein_4001371514',
  'Meat_KSA_Food Services_Protein_4001370150',
  'Meat_KSA_Food Services_Protein_4001362302',
  'Meat_KSA_Food Services_Protein_4001371108',
  'Meat_KSA_Food Services_Protein_5011840107',
  'Meat_KSA_Food Services_Protein_4001371508',
  'Meat_KSA_Food Services_Protein_4001370845',
  'Meat_KSA_Food Services_Protein_4001371509',
  'Meat_KSA_Food Services_Protein_4001370310',
  'Meat_KSA_Food Services_Protein_4001370807',
  'Meat_KSA_Food Services_Protein_4001370140',
  'Meat_KSA_Food Services_Protein_4001350111',
  'Meat_KSA_Food Services_Protein_4001310114',
  'Meat_KSA_Food Services_Protein_4001340101',
  'Meat_KSA_Food Services_Protein_4001340301',
  'Meat_KSA_F

In [35]:
df_no_null_ids_list_all.append('Date')
df_no_null_ids_list_all

['Meat_KSA_Food Services_Protein_4001320101',
 'Meat_KSA_Food Services_Protein_4001330103',
 'Meat_KSA_Food Services_Protein_4001350207',
 'Meat_KSA_Food Services_Protein_4001370144',
 'Meat_KSA_Food Services_Protein_4001370145',
 'Meat_KSA_Food Services_Protein_4001370861',
 'Meat_KSA_Food Services_Protein_4001371514',
 'Meat_KSA_Food Services_Protein_4001370150',
 'Meat_KSA_Food Services_Protein_4001362302',
 'Meat_KSA_Food Services_Protein_4001371108',
 'Meat_KSA_Food Services_Protein_5011840107',
 'Meat_KSA_Food Services_Protein_4001371508',
 'Meat_KSA_Food Services_Protein_4001370845',
 'Meat_KSA_Food Services_Protein_4001371509',
 'Meat_KSA_Food Services_Protein_4001370310',
 'Meat_KSA_Food Services_Protein_4001370807',
 'Meat_KSA_Food Services_Protein_4001370140',
 'Meat_KSA_Food Services_Protein_4001350111',
 'Meat_KSA_Food Services_Protein_4001310114',
 'Meat_KSA_Food Services_Protein_4001340101',
 'Meat_KSA_Food Services_Protein_4001340301',
 'Meat_KSA_Food Services_Protein_4

### Selecting the columns having the largest volume (largest 20)

In [36]:
vol_df_train = pd.DataFrame(df_train.iloc[:, :].sum(axis=0))
vol_df_train = vol_df_train.reset_index()
vol_df_train.columns = ['ID', 'volume']
sorted_vol_df= vol_df_train.sort_values(by= 'volume', ascending= False)
sorted_vol_df_ids = sorted_vol_df.iloc[:20, :]['ID']
sorted_vol_df_ids_20 = list(sorted_vol_df_ids)

In [37]:
sorted_vol_df_ids_20

['Meat_KSA_Wholesale_Protein_4001320201',
 'Meat_KSA_Retail_Protein_4001370861',
 'Meat_KSA_Wholesale_Protein_4001370861',
 'Meat_KSA_Retail_Protein_4001320201',
 'Meat_KSA_Food Services_Protein_4001370813',
 'Meat_KSA_Retail_Protein_4001350114',
 'Meat_KSA_Retail_Protein_4001370124',
 'Meat_KSA_Food Services_Protein_4001370815',
 'Meat_KSA_Wholesale_Protein_4001350114',
 'Meat_KSA_Wholesale_Protein_4001370845',
 'Meat_KSA_Food Services_Protein_4001370861',
 'Meat_KSA_Retail_Protein_4001370872',
 'Meat_KSA_Retail_Protein_4001370145',
 'Meat_KSA_Retail_Protein_4001340501',
 'Meat_KSA_Retail_Protein_4001350111',
 'Meat_KSA_Wholesale_Protein_4001320101',
 'Meat_KSA_Food Services_Protein_4001370845',
 'Meat_KSA_Retail_Protein_4001370144',
 'Meat_KSA_Retail_Protein_4001370859',
 'Meat_KSA_Food Services_Protein_4001370816']

In [38]:
sorted_vol_df_ids_20.append('Date')
sorted_vol_df_ids_20

['Meat_KSA_Wholesale_Protein_4001320201',
 'Meat_KSA_Retail_Protein_4001370861',
 'Meat_KSA_Wholesale_Protein_4001370861',
 'Meat_KSA_Retail_Protein_4001320201',
 'Meat_KSA_Food Services_Protein_4001370813',
 'Meat_KSA_Retail_Protein_4001350114',
 'Meat_KSA_Retail_Protein_4001370124',
 'Meat_KSA_Food Services_Protein_4001370815',
 'Meat_KSA_Wholesale_Protein_4001350114',
 'Meat_KSA_Wholesale_Protein_4001370845',
 'Meat_KSA_Food Services_Protein_4001370861',
 'Meat_KSA_Retail_Protein_4001370872',
 'Meat_KSA_Retail_Protein_4001370145',
 'Meat_KSA_Retail_Protein_4001340501',
 'Meat_KSA_Retail_Protein_4001350111',
 'Meat_KSA_Wholesale_Protein_4001320101',
 'Meat_KSA_Food Services_Protein_4001370845',
 'Meat_KSA_Retail_Protein_4001370144',
 'Meat_KSA_Retail_Protein_4001370859',
 'Meat_KSA_Food Services_Protein_4001370816',
 'Date']

In [39]:
path_20_train = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\Meat_protein_no_nan.csv'

df_20_train = pd.read_csv(path_20_train)
df_20_train = df_20_train.loc[:, sorted_vol_df_ids_20]
df_20_train

,Meat_KSA_Wholesale_Protein_4001320201,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Wholesale_Protein_4001370861,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Wholesale_Protein_4001350114,Meat_KSA_Wholesale_Protein_4001370845,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Wholesale_Protein_4001320101,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Food Services_Protein_4001370816,Date
0,188.697200,226.716000,56.540000,45.770000,96.282900,76.095377,67.990175,80.590000,74.770079,15.577000,41.931000,89.013000,52.495296,28.264400,32.412879,33.320000,23.232000,46.330832,83.037550,8.723040,2020-01-31
1,175.205520,146.170000,74.867000,47.380880,103.463700,76.999443,77.337479,83.370000,64.233479,19.758000,49.445000,57.897333,57.453312,28.899200,28.857292,51.993200,16.983000,22.847310,79.126000,7.066560,2020-02-29
2,404.218800,288.101000,124.497000,135.198320,66.787560,116.475524,101.283061,74.930000,66.805920,13.220000,61.760000,75.580000,81.814656,110.119208,46.454878,55.008000,11.172000,43.360428,133.230450,6.405600,2020-03-31
3,811.824000,128.884700,49.360300,364.888800,16.851420,69.880737,92.049854,2.180000,65.046076,7.580000,15.190000,184.757000,62.383104,204.008000,38.569599,75.320000,2.175000,61.769920,168.958000,0.459000,2020-04-30
4,279.318400,97.161000,80.363000,65.922000,24.188280,34.232662,59.585509,4.020000,46.559599,4.322000,9.733000,166.015000,26.787264,54.218800,21.087138,56.038000,4.770000,30.326451,108.016000,0.122400,2020-05-31
5,135.500000,149.303000,105.909000,51.656800,76.205220,66.743271,89.204166,20.320000,87.838273,8.910000,23.360000,63.203000,78.152256,35.286400,39.127480,41.336000,12.841000,43.869366,62.218000,8.119200,2020-06-30
6,105.929200,64.592000,55.284000,35.994400,64.882200,81.354166,57.032404,29.830000,57.829920,22.074000,38.458000,28.460000,73.695552,18.230800,41.372275,25.188800,25.667000,50.576973,29.134000,40.214520,2020-07-31
7,100.501200,119.536000,48.523000,30.908000,69.987300,64.139484,75.847702,66.478000,52.076581,39.904000,18.172000,20.255000,54.534144,19.087600,39.809919,30.230400,32.295000,37.943772,49.534000,36.774060,2020-08-31
8,158.438800,194.523000,71.064000,50.535600,70.864500,114.205821,83.062226,60.302000,45.165600,39.059000,29.859000,59.056000,75.660480,18.194960,48.085520,38.677200,45.029000,44.042777,36.532000,40.706160,2020-09-30
9,93.766800,146.110000,63.473000,33.231600,65.266740,72.872881,79.281693,82.513000,53.679540,40.088000,28.354000,61.760000,69.034560,25.589200,56.559342,37.392000,32.091000,33.951909,72.660000,35.106360,2020-10-31


In [40]:
# Selecting top 20 volume as 'train data'

# df_20_train.to_csv('train_top_20.csv')

##### --------------------------------------------------------------------
### Working with test data (REFINED)
##### --------------------------------------------------------------------

In [41]:
# Selecting top 20 volume test data 

df_top_20_test= df_no_null_test.loc[:, sorted_vol_df_ids_20]
df_top_20_test

,Meat_KSA_Wholesale_Protein_4001320201,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Wholesale_Protein_4001370861,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Wholesale_Protein_4001350114,Meat_KSA_Wholesale_Protein_4001370845,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Wholesale_Protein_4001320101,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Food Services_Protein_4001370816,Date
0,156.6252,208.830800,119.6470,64.488728,42.13416,31.174320,34.765451,23.341,42.165439,73.25800,64.026,9.3807,11.414323,25.608056,35.901270,43.1392,63.141,15.209513,13.78519,15.58,2023-04-30
1,109.1176,206.594911,208.6110,25.369724,63.53580,36.230477,60.200982,18.380,51.855439,110.27400,88.809,5.4101,31.714220,9.798673,46.799657,14.3028,74.268,43.273197,11.65950,22.38,2023-05-31
2,120.4280,198.975800,161.4146,23.293093,55.44720,35.351281,54.280052,17.680,53.557945,97.60505,102.086,3.3944,23.755066,6.582480,42.361838,32.6020,73.516,41.707308,27.08880,33.57,2023-06-30


In [42]:
# Selecting top 20 volume as 'test data'

# df_top_20_test.to_csv('test_top_20.csv')

In [43]:
# Removing these 2 ID's from train data

'''
['Meat_KSA_Retail_Protein_5011680102', 'Meat_KSA_Retail_Protein_5011680103'] not in index 

'''

"\n['Meat_KSA_Retail_Protein_5011680102', 'Meat_KSA_Retail_Protein_5011680103'] not in index \n\n"

In [44]:
# Selecting all ID's from train data (Number = 200)

df_top_all_test= df_no_null_test.loc[:, df_no_null_ids_list_all]
df_top_all_test

,Meat_KSA_Food Services_Protein_4001320101,Meat_KSA_Food Services_Protein_4001330103,Meat_KSA_Food Services_Protein_4001350207,Meat_KSA_Food Services_Protein_4001370144,Meat_KSA_Food Services_Protein_4001370145,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Food Services_Protein_4001371514,Meat_KSA_Food Services_Protein_4001370150,Meat_KSA_Food Services_Protein_4001362302,Meat_KSA_Food Services_Protein_4001371108,Meat_KSA_Food Services_Protein_5011840107,Meat_KSA_Food Services_Protein_4001371508,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Food Services_Protein_4001371509,Meat_KSA_Food Services_Protein_4001370310,Meat_KSA_Food Services_Protein_4001370807,Meat_KSA_Food Services_Protein_4001370140,Meat_KSA_Food Services_Protein_4001350111,Meat_KSA_Food Services_Protein_4001310114,Meat_KSA_Food Services_Protein_4001340101,Meat_KSA_Food Services_Protein_4001340301,Meat_KSA_Food Services_Protein_4001350602,Meat_KSA_Food Services_Protein_4001351012,Meat_KSA_Food Services_Protein_4001370147,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Food Services_Protein_4001370816,Meat_KSA_Food Services_Protein_4001370889,Meat_KSA_Food Services_Protein_4003350305,Meat_KSA_Food Services_Protein_4003350401,Meat_KSA_Retail_Protein_4001310103,Meat_KSA_Retail_Protein_4001310140,Meat_KSA_Retail_Protein_4001310141,Meat_KSA_Retail_Protein_4001320101,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Retail_Protein_4001330103,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001350207,Meat_KSA_Retail_Protein_4001360102,Meat_KSA_Retail_Protein_4001360106,Meat_KSA_Retail_Protein_4001360308,Meat_KSA_Retail_Protein_4001370123,Meat_KSA_Retail_Protein_4001370125,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001370226,Meat_KSA_Retail_Protein_4001370231,Meat_KSA_Retail_Protein_4001370235,Meat_KSA_Retail_Protein_4001370255,Meat_KSA_Retail_Protein_4001370303,Meat_KSA_Retail_Protein_4001370707,Meat_KSA_Retail_Protein_4001370806,Meat_KSA_Retail_Protein_4001370846,Meat_KSA_Retail_Protein_4001370852,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Retail_Protein_4001370905,Meat_KSA_Retail_Protein_4001371109,Meat_KSA_Retail_Protein_4001371201,Meat_KSA_Retail_Protein_4001371514,Meat_KSA_Retail_Protein_4001370150,Meat_KSA_Retail_Protein_4001370306,Meat_KSA_Retail_Protein_4001370316,Meat_KSA_Retail_Protein_4001370904,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Retail_Protein_4001386130,Meat_KSA_Retail_Protein_4001310174,Meat_KSA_Retail_Protein_4001310402,Meat_KSA_Retail_Protein_4001310505,Meat_KSA_Retail_Protein_4001310701,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001360307,Meat_KSA_Retail_Protein_4001361602,Meat_KSA_Retail_Protein_4001362302,Meat_KSA_Retail_Protein_4001370181,Meat_KSA_Retail_Protein_4001370230,Meat_KSA_Retail_Protein_4001370283,Meat_KSA_Retail_Protein_4001370325,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001371108,Meat_KSA_Retail_Protein_4001371537,Meat_KSA_Retail_Protein_4003350501,Meat_KSA_Retail_Protein_5011810203,Meat_KSA_Retail_Protein_5011840107,Meat_KSA_Retail_Protein_4001310149,Meat_KSA_Retail_Protein_4001371508,Meat_KSA_Retail_Protein_4001350115,Meat_KSA_Retail_Protein_4001370845,Meat_KSA_Retail_Protein_4001371538,Meat_KSA_Retail_Protein_4001370310,Meat_KSA_Retail_Protein_4001370807,Meat_KSA_Retail_Protein_4001310123,Meat_KSA_Retail_Protein_4001370284,Meat_KSA_Retail_Protein_4001370285,Meat_KSA_Retail_Protein_4001370140,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Retail_Protein_4001310506,Meat_KSA_Retail_Protein_4001310114,Meat_KSA_Retail_Protein_4001340101,Meat_KSA_Retail_Protein_4001340301,Meat_KSA_Retail_Protein_4001350602,Meat_KSA_Retail_Protein_4001351201,Meat_KSA_Retail_Protein_4001370147,Meat_KSA_Retail_Protein_4001370272,Meat_KSA_Retail_Protein_4001370273,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Retail_Protein_4003350305,Meat_KSA_Retail_Protein_4003350308,Meat_KSA_Retail_Protein_4003350401,Meat_KSA_

In [45]:
# Selecting ALL volume as 'test data'

# df_top_all_test.to_csv('test_all.csv')

### Concatenating the data

In [46]:
path_train_all = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\train_all.csv'
path_test_all = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\test_all.csv'

path_train_top_20 = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\train_top_20.csv'
path_test_top_20 = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\test_top_20.csv'

In [47]:
df_train_all = pd.read_csv(path_train_all)
df_test_all = pd.read_csv(path_test_all)

df_train_top_20 = pd.read_csv(path_train_top_20)
df_test_top_20 = pd.read_csv(path_test_top_20)

In [48]:
## Setting date as index for all

df_train_all.set_index(['Date'], inplace= True)
df_test_all.set_index(['Date'], inplace= True)
df_train_top_20.set_index(['Date'], inplace= True)
df_test_top_20.set_index(['Date'], inplace= True)

In [54]:
df_test_all = df_test_all.drop(columns= ['Unnamed: 0'])
df_train_top_20 = df_train_top_20.drop(columns= ['Unnamed: 0'])
df_test_top_20 = df_test_top_20.drop(columns= ['Unnamed: 0'])

In [59]:
# Concatenation 

df_all = pd.concat([df_train_all, df_test_all], axis=0)


,Meat_KSA_Food Services_Protein_4001320101,Meat_KSA_Food Services_Protein_4001330103,Meat_KSA_Food Services_Protein_4001350207,Meat_KSA_Food Services_Protein_4001370144,Meat_KSA_Food Services_Protein_4001370145,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Food Services_Protein_4001371514,Meat_KSA_Food Services_Protein_4001370150,Meat_KSA_Food Services_Protein_4001362302,Meat_KSA_Food Services_Protein_4001371108,Meat_KSA_Food Services_Protein_5011840107,Meat_KSA_Food Services_Protein_4001371508,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Food Services_Protein_4001371509,Meat_KSA_Food Services_Protein_4001370310,Meat_KSA_Food Services_Protein_4001370807,Meat_KSA_Food Services_Protein_4001370140,Meat_KSA_Food Services_Protein_4001350111,Meat_KSA_Food Services_Protein_4001310114,Meat_KSA_Food Services_Protein_4001340101,Meat_KSA_Food Services_Protein_4001340301,Meat_KSA_Food Services_Protein_4001350602,Meat_KSA_Food Services_Protein_4001351012,Meat_KSA_Food Services_Protein_4001370147,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Food Services_Protein_4001370816,Meat_KSA_Food Services_Protein_4001370889,Meat_KSA_Food Services_Protein_4003350305,Meat_KSA_Food Services_Protein_4003350401,Meat_KSA_Retail_Protein_4001310103,Meat_KSA_Retail_Protein_4001310140,Meat_KSA_Retail_Protein_4001310141,Meat_KSA_Retail_Protein_4001320101,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Retail_Protein_4001330103,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001350207,Meat_KSA_Retail_Protein_4001360102,Meat_KSA_Retail_Protein_4001360106,Meat_KSA_Retail_Protein_4001360308,Meat_KSA_Retail_Protein_4001370123,Meat_KSA_Retail_Protein_4001370125,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001370226,Meat_KSA_Retail_Protein_4001370231,Meat_KSA_Retail_Protein_4001370235,Meat_KSA_Retail_Protein_4001370255,Meat_KSA_Retail_Protein_4001370303,Meat_KSA_Retail_Protein_4001370707,Meat_KSA_Retail_Protein_4001370806,Meat_KSA_Retail_Protein_4001370846,Meat_KSA_Retail_Protein_4001370852,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Retail_Protein_4001370905,Meat_KSA_Retail_Protein_4001371109,Meat_KSA_Retail_Protein_4001371201,Meat_KSA_Retail_Protein_4001371514,Meat_KSA_Retail_Protein_4001370150,Meat_KSA_Retail_Protein_4001370306,Meat_KSA_Retail_Protein_4001370316,Meat_KSA_Retail_Protein_4001370904,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Retail_Protein_4001386130,Meat_KSA_Retail_Protein_4001310174,Meat_KSA_Retail_Protein_4001310402,Meat_KSA_Retail_Protein_4001310505,Meat_KSA_Retail_Protein_4001310701,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001360307,Meat_KSA_Retail_Protein_4001361602,Meat_KSA_Retail_Protein_4001362302,Meat_KSA_Retail_Protein_4001370181,Meat_KSA_Retail_Protein_4001370230,Meat_KSA_Retail_Protein_4001370283,Meat_KSA_Retail_Protein_4001370325,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001371108,Meat_KSA_Retail_Protein_4001371537,Meat_KSA_Retail_Protein_4003350501,Meat_KSA_Retail_Protein_5011810203,Meat_KSA_Retail_Protein_5011840107,Meat_KSA_Retail_Protein_4001310149,Meat_KSA_Retail_Protein_4001371508,Meat_KSA_Retail_Protein_4001350115,Meat_KSA_Retail_Protein_4001370845,Meat_KSA_Retail_Protein_4001371538,Meat_KSA_Retail_Protein_4001370310,Meat_KSA_Retail_Protein_4001370807,Meat_KSA_Retail_Protein_4001310123,Meat_KSA_Retail_Protein_4001370284,Meat_KSA_Retail_Protein_4001370285,Meat_KSA_Retail_Protein_4001370140,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Retail_Protein_4001310506,Meat_KSA_Retail_Protein_4001310114,Meat_KSA_Retail_Protein_4001340101,Meat_KSA_Retail_Protein_4001340301,Meat_KSA_Retail_Protein_4001350602,Meat_KSA_Retail_Protein_4001351201,Meat_KSA_Retail_Protein_4001370147,Meat_KSA_Retail_Protein_4001370272,Meat_KSA_Retail_Protein_4001370273,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Retail_Protein_4003350305,Meat_KSA_Retail_Protein_4003350308,Meat_KSA_Retail_Protein_4003350401,Meat_KSA_

In [61]:
# df_all.to_csv('Sku_level_vol_all.csv')

In [62]:
# Concatenation 

df_top_20 = pd.concat([df_train_top_20, df_test_top_20], axis=0)

,Meat_KSA_Wholesale_Protein_4001320201,Meat_KSA_Retail_Protein_4001370861,Meat_KSA_Wholesale_Protein_4001370861,Meat_KSA_Retail_Protein_4001320201,Meat_KSA_Food Services_Protein_4001370813,Meat_KSA_Retail_Protein_4001350114,Meat_KSA_Retail_Protein_4001370124,Meat_KSA_Food Services_Protein_4001370815,Meat_KSA_Wholesale_Protein_4001350114,Meat_KSA_Wholesale_Protein_4001370845,Meat_KSA_Food Services_Protein_4001370861,Meat_KSA_Retail_Protein_4001370872,Meat_KSA_Retail_Protein_4001370145,Meat_KSA_Retail_Protein_4001340501,Meat_KSA_Retail_Protein_4001350111,Meat_KSA_Wholesale_Protein_4001320101,Meat_KSA_Food Services_Protein_4001370845,Meat_KSA_Retail_Protein_4001370144,Meat_KSA_Retail_Protein_4001370859,Meat_KSA_Food Services_Protein_4001370816
Date,,,,,,,,,,,,,,,,,,,,
2020-01-31,188.69720,226.7160,56.5400,45.77000,96.28290,76.095377,67.990175,80.59,74.770079,15.577,41.931,89.013000,52.495296,28.264400,32.412879,33.3200,23.232,46.330832,83.03755,8.72304
2020-02-29,175.20552,146.1700,74.8670,47.38088,103.46370,76.999443,77.337479,83.37,64.233479,19.758,49.445,57.897333,57.453312,28.899200,28.857292,51.9932,16.983,22.847310,79.12600,7.06656
2020-03-31,404.21880,288.1010,124.4970,135.19832,66.78756,116.475524,101.283061,74.93,66.805920,13.220,61.760,75.580000,81.814656,110.119208,46.454878,55.0080,11.172,43.360428,133.23045,6.40560
2020-04-30,811.82400,128.8847,49.3603,364.88880,16.85142,69.880737,92.049854,2.18,65.046076,7.580,15.190,184.757000,62.383104,204.008000,38.569599,75.3200,2.175,61.769920,168.95800,0.45900
2020-05-31,279.31840,97.1610,80.3630,65.92200,24.18828,34.232662,59.585509,4.02,46.559599,4.322,9.733,166.015000,26.787264,54.218800,21.087138,56.0380,4.770,30.326451,108.01600,0.12240


In [64]:
# df_top_20.to_csv('Sku_level_vol_top_20.csv')